In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import wilcoxon as wilcoxon
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

# meta analysis functions
import sys
sys.path.append('/code/')
from util import analysis
from util import plotting
from util import util_continuous as utils


In [ ]:
PATH = '/data/'

In [ ]:

with open(PATH + 'encoder-estimation-data/encoder-decoder-data.pickle', 'rb') as handle:
    encoder, encoder_r2, idx_dict, pos_vel_model, pos, dec_vels, decoders = pickle.load(handle)

keys = ['METACPHS_S106', 'METACPHS_S107','METACPHS_S108', 'METACPHS_S109', 'METACPHS_S110', 'METACPHS_S111', 'METACPHS_S112', 'METACPHS_S113', 'METACPHS_S114', 'METACPHS_S115', 'METACPHS_S116', 'METACPHS_S117', 'METACPHS_S118', 'METACPHS_S119']


with open(PATH + 'trial-related-data/times-decoder-update.pickle','rb') as handle:
    update_ix, update_times = pickle.load(handle)
# load the order of the trials in terms of when the subject saw the trial
with open(PATH + 'trial-related-data/decoder_trials_in_order.pickle','rb') as handle:
    trials_list, conds_trial_list = pickle.load(handle)

In [ ]:
# check the encoder shape to match as expected
assert(encoder.shape == (utils.n_blocks, utils.n_keys, utils.n_conds, len(update_ix) - 2, utils.n_ch, 9))

set up the figure

In [ ]:
# check the encoder shape to match as expected
assert(encoder.shape == (utils.n_blocks, utils.n_keys, utils.n_conds, len(update_ix) - 2, utils.n_ch, 9))

In [ ]:
import matplotlib.ticker as ticker

label_size = 6
## SETUP THE FIGURE HERE
## HAVE TO RE-REUN FROM HERE TO "CLEAR" THE PLOT
fig_encdec = plt.figure(figsize = (6.5, 5.5), layout='constrained') # set the total figure size
mosaic = """
    aabb.eeff
    ccdd.gghh
    .........
    iijj.mmnn
    kkll.oopp
    qqqqrrsss
    qqqqrrsss
    """

# this is [ 0,0; 0,1]
#         [ 1,0; 1,1]

# set up the axes
ax_dict = fig_encdec.subplot_mosaic(mosaic)
for ii in ax_dict:
    plotting.remove_and_set_axes(ax_dict[ii], bottom=True, left=True)
    ax_dict[ii].tick_params(axis='both', which='major', labelsize = label_size)
    ax_dict[ii].tick_params(axis='both', which='minor', labelsize = label_size)
    # set the x-ticks at multiples of 1
    ax_dict[ii].xaxis.set_major_locator(ticker.MultipleLocator(base=5))
    ax_dict[ii].xaxis.set_minor_locator(ticker.MultipleLocator(base=1))
fig_encdec.patch.set_facecolor('white')


ax_dict['a'].set_title("$D \cdot F_0$")


ax_dict['a'].set_ylabel("(0, 0)")
ax_dict['b'].set_ylabel("(0, 1)")
ax_dict['c'].set_ylabel("(1, 0)")
ax_dict['d'].set_ylabel("(1, 1)")

ax_dict['e'].set_title("$D \cdot F_1$")

ax_dict['e'].set_ylabel("(0, 0)")
ax_dict['f'].set_ylabel("(0, 1)")
ax_dict['g'].set_ylabel("(1, 0)")
ax_dict['h'].set_ylabel("(1, 1)")

ax_dict['i'].set_title("$D \cdot B_0$")

ax_dict['i'].set_ylabel("(0, 0)")
ax_dict['j'].set_ylabel("(0, 1)")
ax_dict['k'].set_ylabel("(1, 0)")
ax_dict['l'].set_ylabel("(1, 1)")

ax_dict['m'].set_title("$D \cdot B_1$")

ax_dict['m'].set_ylabel("(0, 0)")
ax_dict['n'].set_ylabel("(0, 1)")
ax_dict['o'].set_ylabel("(1, 0)")
ax_dict['p'].set_ylabel("(1, 1)")

ax_dict['q'].set_xlabel("Across Trials")
ax_dict['q'].set_ylabel(r'$|E_f - E_i|_F$')

ax_dict['s'].set_xlabel("Time (min)")
ax_dict['s'].set_ylabel(r'$|E_{t+1} - E_t|_F$')


Table of variables used 

| Variable | Shape | Shape Representations | Meaning |
| --- | --- | --- | --- |
| encoder_linear | (2, 14, 8, 17, 64, 8) | blocks x subjects x conditions x number of updates x number of channels x number of encoder features | the encoder estimations of each user and each trial without the affine term |
|enc_all_min1 - 5 | (2, 14, 8, 64, 8) | blocks x subjects x conditions x number of channels x number of encoder features | the encoder estimations of each user and each trial averaged per minute of trial to dampen the overfitting that might have happened |
| enc_all_diff | (4, 2, 14, 8) | interval between minutes in 5-minute trial x blocks x subjects x conditions | the frobenious norm of the difference between each minute-averaged encoder = fro norm(E_{t+1} - E_{t}) |
| enc_all_fi | (2, 14, 8)| last minute - first minute (1) x blocks x subjects x conditions | the frobenious norm of the difference between the first and last-averaged encoder = fro norm(E_{final} - E_{inital}) |
| enc_all_diff_subj | (14, ) | subjects | mean of enc_all_diff per subject |
| enc_fi_diff_subj | (14, ) | subjects | mean of enc_fi_diff per subject |



Explanations of shapes:
* number of blocks = 2 -- experimental setup
* numebr of subject = 14 -- experimental setup
* number of conditions = 8 -- experimental setup of decoder conditions
* number of updates = 17 -- this is the number of decoder updates - 2 (why -2? because we have to take the sifference between updates and the last one was cut short)
* number of channels = 64 -- the number of emg channels and the number of channels used for the decoder, so can keep the multiplication
* number of features = 8 -- the encoder has 8 features and 1 affine term; the feature are: target position (x, y), target velocity (x, y), position error (x, y), velocity error (x, y)

In [ ]:
## this code sets up the encoder variables to be plotted and compared in this notebook

## set up variables to use for this code
# number of features of the encoder
# 8 features of the encoder = target position (x, y), target velocity (x, y), position error (x, y), velocity error (x, y)
n_feat = 8 

## take the encoder variables without the affine term 
# this is done because the affine term was to fit the encoder, but does not represent any encoding of the user
encoder_linear = encoder[:, :, :, :, :, :-1]
# check that encoder_linear variable is the right shape -- same as the encoder, but the last axis should be the number of features
assert(encoder_linear.shape == (utils.n_blocks, utils.n_keys, utils.n_conds, len(update_ix) - 2, utils.n_ch, n_feat))

## average the encoder across minute to use the minute-averaged encoder as a measurement of the user
# this averaging is done so that we can reduce the overfitting of the encoder to the 20-second intervals
axis_time = 3 # the time axis is the 4th one, so = 3
enc_all_min1 = np.mean(encoder_linear[:, :, :, 1:4], axis = axis_time) # average across the first minute
enc_all_min2 = np.mean(encoder_linear[:, :, :, 4:7], axis = axis_time) # average across the 2nd minute
enc_all_min3 = np.mean(encoder_linear[:, :, :, 7:10], axis = axis_time) # average across the 3rd minute
enc_all_min4 = np.mean(encoder_linear[:, :, :, 10:13], axis = axis_time) # average across the 4th minute
enc_all_min5 = np.mean(encoder_linear[:, :, :, -3:], axis = axis_time) # average across the last minute
assert(enc_all_min1.shape == enc_all_min2.shape == enc_all_min3.shape 
       == enc_all_min4.shape == enc_all_min5.shape 
       == (utils.n_blocks, utils.n_keys, utils.n_conds, utils.n_ch, n_feat))

# stack the minute-averaged encoders together so we can find the change in the users' encoders minute to minute
enc_all_min = np.stack((enc_all_min1, enc_all_min2, enc_all_min3, enc_all_min4, enc_all_min5))
# 5 in the first axis since we're stacking 5 arrays together
assert(enc_all_min.shape == (5, utils.n_blocks, utils.n_keys, utils.n_conds, utils.n_ch,  n_feat))

## subtract the difference in the encoder from minute to minute and then take the frobenious norm of that difference
# this is |E_t+1 - E_t|_F
enc_all_diff = np.linalg.norm(np.diff(enc_all_min, axis = 0), axis = (-1, -2))
# this is |E_final - E_initial|_F
enc_all_fi = np.linalg.norm(enc_all_min5 - enc_all_min1, axis = (-1, -2))

# find the mean difference per subject
enc_all_diff_subj = np.mean(enc_all_diff, axis = (0, 1, 3))
enc_fi_diff_subj = np.mean(enc_all_fi, axis = (0, 2))
assert(enc_all_diff_subj.shape == enc_fi_diff_subj.shape == (utils.n_keys, ))

In [ ]:
enc_all_fi.shape

In [ ]:
data1 = np.ndarray.flatten(enc_fi_diff_subj)
data2 = np.ndarray.flatten(enc_all_diff_subj)
data1.shape

In [ ]:
# fig,ax = plt.subplots(1,1,figsize=(3, 5))
fig_ax = ax_dict['r']

data1 = np.ndarray.flatten(enc_fi_diff_subj)
data2 = np.ndarray.flatten(enc_all_diff_subj)
assert(data1.shape == data2.shape == (utils.n_keys,))
data_groups = [data1, data2]
data_labels = ['$|E_f-E_i|$', '$|E_{t+1} - E_t|$']
data_pos = [0, 0.4]

bplot = fig_ax.boxplot(data_groups, 
                    showfliers=False,
                    patch_artist=True,
                    positions=data_pos,
                    widths = 0.3,
                    boxprops=dict(edgecolor="white", alpha=0.5),
                    medianprops=dict(color='k', lw=0.5))


for patch, color in zip(bplot['boxes'], [utils.colors['E'], utils.colors['E']]):
    patch.set_facecolor(color)


# rotate labels  
fig_ax.set_xticks(data_pos,data_labels, rotation=0)
w = wilcoxon(data1, data2) 
print(w)

plotting.plot_significance(pvalue = w.pvalue, data1=data1, data2 = data2, data_pos = data_pos, 
                  fig=fig_encdec, ax=fig_ax, fontsize=10, lw=0.5, y_asterix=0.6, y_bar=0.5)

fig_encdec

In [ ]:
enc_change_across_trials = np.zeros((utils.n_blocks, utils.n_keys, utils.n_conds))
trial_label = np.zeros((utils.n_blocks, utils.n_keys, utils.n_conds))
x_ticks = np.arange(1, 17, 1)  # Ticks for every integer from 0 to 14
f, ax = plt.subplots(utils.n_keys, 1, figsize=(2, 10), layout = 'constrained')
for iK in range(utils.n_keys):
    # x_ticks = np.arange(1, 17, 1)  # Ticks for every integer from 0 to 14
    # block 1
    iB = 0
    # for iC in conds_trial_list[iB, iK]:
    enc_diff_trial = (np.linalg.norm(enc_all_min5[iB, iK, conds_trial_list[iB, iK]] - enc_all_min1[iB, iK, conds_trial_list[iB, iK]], axis = (-1, -2)))
                    #   /np.linalg.norm(enc_all_init[iB, iK, conds_trial_list[iB, iK]], axis = (-1, -2)))
    enc_change_across_trials[iB, iK, :] = enc_diff_trial
    # ax[iK].plot(x_ticks[:8], enc_all_mean[iB, iK, conds_trial_list[iB, iK]], color = 'blue')
    iB = 1
    enc_diff_trial = (np.linalg.norm(enc_all_min5[iB, iK, conds_trial_list[iB, iK]] - enc_all_min1[iB, iK, conds_trial_list[iB, iK]], axis = (-1, -2)))
                        # /np.linalg.norm(enc_all_init[iB, iK, conds_trial_list[iB, iK]], axis = (-1, -2)))
    enc_change_across_trials[iB, iK, :] = enc_diff_trial
    data_to_plot = np.append(enc_change_across_trials[0, iK], enc_change_across_trials[1, iK])
    
    ax[iK].plot(x_ticks, data_to_plot, '-o', color = utils.colors['E'], ms = 1, lw = 1)
    # ax[iK].plot(x_ticks[8:], enc_all_mean[iB, iK, conds_trial_list[iB, iK]], color = 'blue')

    # ax[iK].plot(np.linspace(9, 16, 8), enc_change_across_trials[1, iK], '-o', color = 'darkgray')
    # Set x-axis ticks and labels
    x_labels = [str(i) if i % 2 == 0 else "" for i in x_ticks]  # Labels for even integers, empty string for odd ones
    ax[iK].set_xticks(x_ticks, x_labels, fontsize = 4)
    # ax[iK].vlines(x = 8, ymin=-0.5, ymax = max(data_to_plot), zorder = 0)

    # # y-axis
    ax[iK].set_ylabel(r'$|E_f - E_i|_F$', fontsize = 4)
    ax[iK].set_title(keys[iK], fontsize = 4)
    enc_mean = np.mean(enc_all_diff[:, :, iK].flatten())
    enc_std = np.std(enc_all_diff[:, :, iK].flatten())
    ax[iK].hlines(y = enc_mean, xmin = 1, xmax = 16, zorder = 0, ls = '--', lw = 1)
    # ax[iK].hlines(y = enc_null_std[iK], xmin = 0, xmax = 16, zorder = 0)
    ax[iK].fill_between(x_ticks, y1 = enc_mean - enc_std, 
                        y2 = enc_mean + enc_std, alpha = 0.2, color = utils.colors['E'])
    ax[iK].tick_params(axis='both', which='major', labelsize = label_size)
    ax[iK].tick_params(axis='both', which='minor', labelsize = label_size)
    # plt.show()

# f.set_size_inches(6, 15)

Encoder-Decoder Pairs

In [ ]:
enc_dim = encoder[:, :, :, :, :, idx_dict['targ_pos_idx']] # use reference only
enc_dim.shape
t_upd = update_times[1:-1]

In [ ]:
# target position contribution
# D * F0
enc_dim = encoder_linear[:, :, :, :, :, idx_dict['targ_pos_idx']] # use reference only

# calculate the decoder-encoder pairs here
enc_dec = np.zeros((utils.n_blocks, utils.n_keys, utils.n_conds, len(update_ix) - 2, 2, 2))
enc_dec_control = np.zeros_like((enc_dec))

## takes a long time to run!
for iK, key in enumerate(keys):
    # block 1
    b = 0
    dec1 = decoders[b, iK][:, 1:-1]
    enc1 = enc_dim[b, iK] # [:, np.newaxis]
    enc_dec[b, iK] = dec1@enc1

    # # control - mean of each user's encoder
    # # assume the encoder of each user did not change across blocks, conditions, or within trial
    # enc_cont_subj = enc_control[iK]
    # enc_dec_control[b, iK] = dec1@enc_cont_subj
    
    # block 1
    b = 1
    dec2 = decoders[b, iK][:, 1:-1]
    enc2 = enc_dim[b, iK] #[:, np.newaxis]
    enc_dec[b, iK] = dec2@enc2

    # # control - mean of each user's encoder
    # enc_dec_control[b, iK] = dec2@enc_cont_subj

ed_all = np.mean(enc_dec, axis = (0, 2))
ed_all_control = np.mean(enc_dec_control, axis = (0, 2))

# ld_low
# [a  b]
# [c  d]
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 0, 0], axis = 0, ax=ax_dict['a'], 
                          color = utils.colors['FF'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 0, 1], axis = 0, ax=ax_dict['b'], 
                          color = utils.colors['FF'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 1, 0], axis = 0, ax=ax_dict['c'],
                          color = utils.colors['FF'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 1, 1], axis = 0, ax=ax_dict['d'], 
                          color = utils.colors['FF'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)

# set idealized lines
ax_dict['a'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=0)
ax_dict['b'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=0)
ax_dict['c'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=0)
ax_dict['d'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=0)

ax_dict['a'].get_shared_y_axes().join(ax_dict['a'], ax_dict['b'], ax_dict['c'], ax_dict['d'])
ax_dict['b'].get_shared_y_axes().join(ax_dict['a'], ax_dict['b'], ax_dict['c'], ax_dict['d'])

print("mean values of the DF0 matrix: ", np.mean(ed_all[:, -3:], axis = (0, 1)))
print("std values of the DF0 matrix: ", np.std(ed_all[:, -3:], axis = (0, 1)))
fig_encdec


In [ ]:
# target velocity contribution

enc_dim = encoder_linear[:, :, :, :, :, idx_dict['targ_vel_idx']] # use reference only


# calculate the decoder-encoder pairs here
enc_dec = np.zeros((utils.n_blocks, utils.n_keys, utils.n_conds, len(update_ix) - 2, 2, 2))
enc_dec_control = np.zeros_like((enc_dec))

## takes a long time to run!
for iK, key in enumerate(keys):
    # block 1
    b = 0
    dec1 = decoders[b, iK][:, 1:-1]
    enc1 = enc_dim[b, iK] # [:, np.newaxis]
    enc_dec[b, iK] = dec1@enc1

    # block 1
    b = 1
    dec2 = decoders[b, iK][:, 1:-1]
    enc2 = enc_dim[b, iK] #[:, np.newaxis]
    enc_dec[b, iK] = dec2@enc2


ed_all = np.mean(enc_dec, axis = (0, 2))
ed_all_control = np.mean(enc_dec_control, axis = (0, 2))

# ld_low
# [e  f]
# [g  h]
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 0, 0], axis = 0, ax=ax_dict['e'], 
                          color = utils.colors['FF'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 0, 1], axis = 0, ax=ax_dict['f'], 
                          color = utils.colors['FF'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 1, 0], axis = 0, ax=ax_dict['g'],
                          color = utils.colors['FF'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 1, 1], axis = 0, ax=ax_dict['h'], 
                          color = utils.colors['FF'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)


# set idealized lines
ax_dict['e'].axhline(y=1,xmin=0,xmax=5,c='k',ls = '--', linewidth=1,zorder=0)
ax_dict['f'].axhline(y=0,xmin=0,xmax=5,c='k',ls = '--', linewidth=1,zorder=0)
ax_dict['g'].axhline(y=0,xmin=0,xmax=5,c='k',ls = '--', linewidth=1,zorder=0)
ax_dict['h'].axhline(y=1,xmin=0,xmax=5,c='k',ls = '--', linewidth=1,zorder=0)

# share y-axis
ax_dict['e'].get_shared_y_axes().join(ax_dict['e'], ax_dict['f'], ax_dict['g'], ax_dict['h'])
ax_dict['f'].get_shared_y_axes().join(ax_dict['e'], ax_dict['f'], ax_dict['g'], ax_dict['h'])


print("mean values of the DF1 matrix: ", np.mean(ed_all[:, -3:], axis = (0, 1)))
print("std values of the DF1 matrix: ", np.std(ed_all[:, -3:], axis = (0, 1)))

fig_encdec


In [ ]:
# target position error contribution - DB_0

enc_dim = encoder_linear[:, :, :, :, :, idx_dict['targ_pos_err_idx']] # use reference only


# calculate the decoder-encoder pairs here
enc_dec = np.zeros((utils.n_blocks, utils.n_keys, utils.n_conds, len(update_ix) - 2, 2, 2))
enc_dec_control = np.zeros_like((enc_dec))

## takes a long time to run!
for iK, key in enumerate(keys):
    # block 1
    b = 0
    dec1 = decoders[b, iK][:, 1:-1]
    enc1 = enc_dim[b, iK] # [:, np.newaxis]
    enc_dec[b, iK] = dec1@enc1

    # block 1
    b = 1
    dec2 = decoders[b, iK][:, 1:-1]
    enc2 = enc_dim[b, iK] #[:, np.newaxis]
    enc_dec[b, iK] = dec2@enc2


ed_all = np.mean(enc_dec, axis = (0, 2))
ed_all_control = np.mean(enc_dec_control, axis = (0, 2))

# ld_low
# [ i  j]
# [ k  l]
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 0, 0], axis = 0, ax=ax_dict['i'], 
                          color = utils.colors['FB'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 0, 1], axis = 0, ax=ax_dict['j'], 
                          color = utils.colors['FB'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 1, 0], axis = 0, ax=ax_dict['k'],
                          color = utils.colors['FB'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 1, 1], axis = 0, ax=ax_dict['l'], 
                          color = utils.colors['FB'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)

# idealized values
ax_dict['j'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=10)
ax_dict['k'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=10)

# share y-axis
ax_dict['j'].get_shared_y_axes().join(ax_dict['i'], ax_dict['j'], ax_dict['k'], ax_dict['l'])

print("mean values of the DB0 matrix: ", np.mean(ed_all[:, -3:], axis = (0, 1)))
print("std values of the DB0 matrix: ", np.std(ed_all[:, -3:], axis = (0, 1)))
print("eigen values of the DB0 matrix: ", np.linalg.eigvals(np.mean(ed_all[:, -3:], axis = (0, 1))))

fig_encdec

In [ ]:
# target velocity error contribution

enc_dim = encoder_linear[:, :, :, :, :, idx_dict['targ_vel_err_idx']] # use reference only


# calculate the decoder-encoder pairs here
enc_dec = np.zeros((utils.n_blocks, utils.n_keys, utils.n_conds, len(update_ix) - 2, 2, 2))
enc_dec_control = np.zeros_like((enc_dec))

## takes a long time to run!
for iK, key in enumerate(keys):
    # block 1
    b = 0
    dec1 = decoders[b, iK][:, 1:-1]
    enc1 = enc_dim[b, iK] # [:, np.newaxis]
    enc_dec[b, iK] = dec1@enc1

    # block 1
    b = 1
    dec2 = decoders[b, iK][:, 1:-1]
    enc2 = enc_dim[b, iK] #[:, np.newaxis]
    enc_dec[b, iK] = dec2@enc2


ed_all = np.mean(enc_dec, axis = (0, 2))

ed_all_control = np.mean(enc_dec_control, axis = (0, 2))

# ld_low
# [ m  n]
# [ o  p]
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 0, 0], axis = 0, ax=ax_dict['m'], 
                          color = utils.colors['FB'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 0, 1], axis = 0, ax=ax_dict['n'], 
                          color = utils.colors['FB'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 1, 0], axis = 0, ax=ax_dict['o'],
                          color = utils.colors['FB'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)
plotting.plot_time_domain(t_upd/60, ed_all[:, :, 1, 1], axis = 0, ax=ax_dict['p'], 
                          color = utils.colors['FB'], ls='-o', alpha = 0.1, lw=1, ms = 2, remove_axes=False)

# idealized values
ax_dict['n'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=10)
ax_dict['o'].axhline(y=0,xmin=0,xmax=5,c="black",ls = '--', linewidth=1,zorder=10)

# share y-axis
ax_dict['n'].get_shared_y_axes().join(ax_dict['m'], ax_dict['n'], ax_dict['o'], ax_dict['p'])

print("mean values of the DB1 matrix: ", np.mean(ed_all[:, -3:], axis = (0, 1)))
print("std values of the DB1 matrix: ", np.std(ed_all[:, -3:], axis = (0, 1)))
print("eigen values of the DB1 matrix: ", np.linalg.eigvals(np.mean(ed_all[:, -3:], axis = (0, 1))))


fig_encdec

In [ ]:
# fig q
ex_subj = 1
ex_enc_change = np.append(enc_change_across_trials[0, ex_subj], enc_change_across_trials[1, ex_subj])
ex_enc_change_all = np.append(enc_change_across_trials[0, :], enc_change_across_trials[1, :], axis = -1)

plotting.plot_time_domain(x_ticks, ex_enc_change_all, axis = 0,
                          lw = 1, ls = '-o', ms = 2,
                          remove_axes=False, ax = ax_dict['q'], color=utils.colors['E'])
fig_encdec

In [ ]:
encoder_linear.shape

In [ ]:
E_diff = np.linalg.norm(np.diff(encoder_linear, axis = 3), 'fro', axis = (-1, -2))
E_diff.shape

In [ ]:
E_diff = np.linalg.norm(np.diff(encoder_linear, axis = 3), 'fro', axis = (-1, -2))
E_diff_subj = np.mean(E_diff, axis = (0, 2))

plotting.plot_time_domain(np.linspace(0.6, 5, 15), E_diff_subj[:, 1:], axis = 0,
                          lw = 1, ls = '-o', ms = 2,
                          remove_axes=False, ax = ax_dict['s'], color=utils.colors['E'])

# plotting.plot_time_domain(np.linspace(0.6, 5, 16), E_diff_subj[:, :], axis = 0,
#                           lw = 1, ls = '-o', ms = 2,
#                           remove_axes=False, ax = ax_dict['s'], color=utils.colors['E'])

# ax_dict['s'].plot(E_diff_subj[:, 1:].T, lw = 1)
fig_encdec
# plt.plot(E_diff_subj[:, 1:].T)

In [ ]:
image_format = 'pdf' # e.g .png, .svg, etc.
image_name = 'fig2-encoder-decoder-mean.pdf'

PATH = '/results/'
fig_encdec.savefig(PATH + image_name, format=image_format, dpi=300)